<a href="https://colab.research.google.com/github/linhnguyen222/DPS-Silo/blob/master/pretrained_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Pre trained BERT model for full sentence multiclass classification


In [ ]:
from platform import python_version
from pathlib import Path
import tensorflow as tf
import numpy as np

import pathlib
import pandas as pd
#from pytorch_pretrained_bert import BertTokenizer
import torch
from pandas import DataFrame


print(python_version())
print (tf.__version__)

3.6.9
2.3.0


In [ ]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


####Helper functions to read in the data.

In [ ]:
def read_data(directory):
    ids = []
    texts = []
    labels = []
    for f in directory.glob('*.txt'):
        id = f.name.replace('article', '').replace('.txt','')
        ids.append(id)
        texts.append(f.read_text())
        labels.append(parse_label(f.as_posix().replace('.txt', '.labels.tsv')))
    # labels can be empty 
    return ids, texts, labels

def clean_text(articles, ids):
    texts = []
    for article, id in zip(articles, ids):
        sentences = article.split('\n')
        start = 0
        end = -1
        res = []
        for sentence in sentences:
           start = end + 1
           end = start + len(sentence)  # length of sequence 
           if sentence != "": # if not empty line
               res.append([id, sentence, start, end])
        texts.append(res)
    return texts


def make_dataset(directory):
    ids, texts, labels = read_data(directory)
    texts = clean_text(texts, ids)
    res = []
    for text, label in zip(texts, labels):
        # making positive examples
        tmp = [] 
        pos_ind = [0] * len(text)
        for l in label:
            for i, sen in enumerate(text):
                if l[0] >= sen[2] and l[0] < sen[3] and l[1] > sen[3]:
                    l[4] = 1
                    tmp.append(sen + [l[0], sen[3], l[2], l[3], l[4]])
                    pos_ind[i] = 1
                    l[0] = sen[3] + 1
                elif l[0] != l[1] and l[0] >= sen[2] and l[0] < sen[3] and l[1] <= sen[3]: 
                    tmp.append(sen + l)
                    pos_ind[i] = 1
        # making negative examples
        dummy = [0, 0, 'O', 0, 0]
        for k, sen in enumerate(text):
            if pos_ind[k] != 1:
                tmp.append(sen+dummy)
        res.append(tmp)     
    return res
  
def parse_label(label_path):
    labels = []
    f= Path(label_path)
    
    if not f.exists():
        return labels

    for line in open(label_path):
        parts = line.strip().split('\t')
        labels.append([int(parts[2]), int(parts[3]), parts[1], 0, 0])
    labels = sorted(labels) 

    if labels:
        length = max([label[1] for label in labels]) 
        visit = np.zeros(length)
        res = []
        for label in labels:
            if sum(visit[label[0]:label[1]]):
                label[3] = 1
            else:
               visit[label[0]:label[1]] = 1
            res.append(label)
        return res 
    else:
        return labels

####Importing train and test set and reading them in as dataframes.

In [ ]:
#train set
dataset_train=make_dataset(Path('/content/drive/My Drive/DSP/data/protechn_corpus_eval/train'))
#test set
dataset_test=make_dataset(Path('/content/drive/My Drive/DSP/data/protechn_corpus_eval/test'))


In [ ]:
#train dataframe
empty=[]
for i in dataset_train:
    
    temp=DataFrame(i, columns=['id', 'full_sent', 'start_sent', 'end_sent', 'start_prop', 'end_prop','prop', 'ex1', 'ex2' ])
    empty.append(temp)

df_train=pd.concat(empty)  
df_train = df_train.drop(['ex1'],axis =1).drop(['ex2'],axis =1)
df_train.prop.value_counts()
df_train=df_train.reset_index()
#del df['index']

df_train.head()

,index,id,full_sent,start_sent,end_sent,start_prop,end_prop,prop
0,0,7618745059,“They interpreted the law in my case to say it...,1695,1873,1831,1872,Whataboutism
1,1,7618745059,Two guys on my ship did the same thing and wer...,1905,1977,1925,1976,Whataboutism
2,2,7618745059,“They used me as an example because of [the ba...,2312,2454,2312,2388,Causal_Oversimplification
3,3,7618745059,The government actively destroyed his life an ...,2710,2819,2725,2782,"Exaggeration,Minimisation"
4,4,7618745059,Ex-Sailor Pardoned By Trump Says He’s SUING Ob...,0,59,0,0,O


#### We define a new column (called 'label') to turn string labels into integer labels by using a defined label dictionary that maps each propaganda technique to an integer representing that class.

In [ ]:
"""
label_dict={}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label]=index
print(label_dict)
"""
label_dict={'O': 0, 'Black-and-White_Fallacy': 1, 'Loaded_Language': 2, 'Flag-Waving': 3, 
 'Name_Calling,Labeling': 4, 'Slogans': 5, 'Causal_Oversimplification': 6, 'Whataboutism': 7,
 'Exaggeration,Minimisation': 8, 'Doubt': 9, 'Appeal_to_Authority': 10, 'Repetition': 11, 'Appeal_to_fear-prejudice': 12,
 'Thought-terminating_Cliches': 13, 'Bandwagon': 14, 'Red_Herring': 15, 'Reductio_ad_hitlerum': 16,
 'Obfuscation,Intentional_Vagueness,Confusion': 17, 'Straw_Men': 18}

df_train['label']=df_train.prop.replace(label_dict)

In [ ]:
df_train.head()

,index,id,full_sent,start_sent,end_sent,start_prop,end_prop,prop,label
0,0,7618745059,“They interpreted the law in my case to say it...,1695,1873,1831,1872,Whataboutism,7
1,1,7618745059,Two guys on my ship did the same thing and wer...,1905,1977,1925,1976,Whataboutism,7
2,2,7618745059,“They used me as an example because of [the ba...,2312,2454,2312,2388,Causal_Oversimplification,6
3,3,7618745059,The government actively destroyed his life an ...,2710,2819,2725,2782,"Exaggeration,Minimisation",8
4,4,7618745059,Ex-Sailor Pardoned By Trump Says He’s SUING Ob...,0,59,0,0,O,0


In [ ]:
df_train.dtypes


index          int64
id            object
full_sent     object
start_sent     int64
end_sent       int64
start_prop     int64
end_prop       int64
prop          object
label          int64
dtype: object

In [ ]:
type(df_train['full_sent'][0])

str

####Dataset is imbalanced.

In [ ]:
#data is not unitedly distributed in different classes i.e., we have a class imbalance
df_train.prop.value_counts()

O                                              10337
Loaded_Language                                 1832
Name_Calling,Labeling                            934
Doubt                                            527
Repetition                                       465
Exaggeration,Minimisation                        401
Appeal_to_fear-prejudice                         225
Flag-Waving                                      216
Causal_Oversimplification                        199
Appeal_to_Authority                              129
Black-and-White_Fallacy                          123
Slogans                                          123
Thought-terminating_Cliches                       72
Whataboutism                                      59
Reductio_ad_hitlerum                              48
Red_Herring                                       26
Bandwagon                                         13
Straw_Men                                         12
Obfuscation,Intentional_Vagueness,Confusion   

In [ ]:
df_train.label.value_counts()

0     10337
2      1832
4       934
9       527
11      465
8       401
12      225
3       216
6       199
10      129
5       123
1       123
13       72
7        59
16       48
15       26
14       13
18       12
17       11
Name: label, dtype: int64

In [ ]:
#test dataframe
empty=[]
for i in dataset_test:
    
    temp=DataFrame(i, columns=['id', 'full_sent', 'start_sent', 'end_sent', 'start_prop', 'end_prop','prop', 'ex1', 'ex2' ])
    empty.append(temp)

df_test=pd.concat(empty)  
df_test = df_test.drop(['ex1'],axis =1).drop(['ex2'],axis =1)
df_test.prop.value_counts()
df_test=df_test.reset_index()
#del df['index']

df_test.head()


,index,id,full_sent,start_sent,end_sent,start_prop,end_prop,prop
0,0,111111133,"CNN in turn dropped its lawsuit on the matter,...",301,415,358,367,Loaded_Language
1,1,111111133,But while it yielded to Mr. Acosta — whose tes...,417,673,460,465,Loaded_Language
2,2,111111133,But while it yielded to Mr. Acosta — whose tes...,417,673,504,507,Loaded_Language
3,3,111111133,The White House sought to blame Mr. Acosta for...,967,1167,1070,1077,Loaded_Language
4,4,111111133,Codifying the behavior of journalists struck s...,1168,1391,1224,1244,Loaded_Language


In [ ]:

df_test['label']=df_test.prop.replace(label_dict)

In [ ]:
df_test.head()

,index,id,full_sent,start_sent,end_sent,start_prop,end_prop,prop,label
0,0,111111133,"CNN in turn dropped its lawsuit on the matter,...",301,415,358,367,Loaded_Language,2
1,1,111111133,But while it yielded to Mr. Acosta — whose tes...,417,673,460,465,Loaded_Language,2
2,2,111111133,But while it yielded to Mr. Acosta — whose tes...,417,673,504,507,Loaded_Language,2
3,3,111111133,The White House sought to blame Mr. Acosta for...,967,1167,1070,1077,Loaded_Language,2
4,4,111111133,Codifying the behavior of journalists struck s...,1168,1391,1224,1244,Loaded_Language,2




####If we wanted to test this model with Augmented Data then we would be running these lines.



In [ ]:
print("size of train set is: ", df_train.shape[0])
print("size of test set is: ", df_test.shape[0])

size of train set is:  15752
size of test set is:  4490


In [ ]:
#df_new=pd.read_csv(Path("/content/ADD_Context.csv"))
#df_new.head()
#df_new['Sentence'].isnull()
#df_new.isnull().values.any()
#df_new = df_new.dropna()
#df_new.isnull().values.any()
#df_new['label']=df_new.Label.replace(label_dict)
#df_new = df_new.rename(columns = {'Sentence':'full_sent'})

In [ ]:
#df_train=df_new.iloc[0:20000, 0:5]

In [ ]:
#df_train=df_train.reset_index()


In [ ]:
#df_test=df_new.iloc[20000:, 0:5]

In [ ]:
!pip install transformers



###The tokenizer takes in raw text and splits it into tokens. A token in this case is a numerical number that represents a certain word. Tensordataset makes it possible to use the data set in a pytorch environment. 


In [ ]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

###We use a pretrained BERT to encode our dataset. 



In [ ]:
#we will be using all lower case data
#do_lower_case converts every string to lower case as an uncased version of BERT will be used
tokenizer= BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


####Now we need to convert all our sentences from language into encoded form. Batch encode class can perform this as it can take multiple strings and convert them into tokens as is needed. This is done separately for train and test data. 


In [ ]:
""" As parameters it takes actual sentences, add special tokens will highlight when a sentences ends and when a new one begins, 
an attention mask tells the model where the relevant information in a sentence lies - as sentences do not originaly have the same length """

encoded_data_train=tokenizer.batch_encode_plus(df_train.full_sent.values, add_special_tokens=True, 
                                               return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')

encoded_data_test=tokenizer.batch_encode_plus(df_test.full_sent.values, add_special_tokens=True, 
                                               return_attention_mask=True, pad_to_max_length=True, max_length=256, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
""" access the data train dictionary and pull out the input ids which essetntailly represents each word as a number"""
input_ids_train=encoded_data_train['input_ids']
attention_masks_train=encoded_data_train['attention_mask']
#make a tensor out of the original data labels
labels_train=torch.tensor(df_train.label.values, dtype=torch.long)



####We create the train data set by TensorDataset which used in pytorch library.



In [ ]:
#iterates over each three element at one time for each data set
dataset_train=TensorDataset(input_ids_train,
                            attention_masks_train,labels_train)


In [ ]:
#repeat for test set
input_ids_test=encoded_data_test['input_ids']
attention_masks_test=encoded_data_test['attention_mask']
labels_test=torch.tensor(df_test.label.values, dtype=torch.long)

In [ ]:
dataset_test=TensorDataset(input_ids_test,
                            attention_masks_test,labels_test)


In [ ]:
print(len(dataset_train))
print(len(dataset_test))

15752
4490


####We import the pretrained BERT model from module Huggingface's transformer library.

In [ ]:
from transformers import BertForSequenceClassification

####In this part we can as well fine tune the pretrained BERT by outlining number of labels the final layer of BERT should have. We add a layer on top of that of size 19 as a classifier. 


In [ ]:
model= BertForSequenceClassification.from_pretrained(
                'bert-base-uncased',
                #length of the label dictionary is 19
                 num_labels=19, 
                  output_attentions=False, 
                  #the state before the output state which won't be needed to return
                 output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

####Dataloaders make it possible to iterate over the dataset in batches. Randomsampler samples the data randomly per batch. This will help randomize what data the model is exposed to. Sequential sampler can be used for test set, as for test set randomly sorting the data set is not as important as the gradients are fixed already in that stage.

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [ ]:
batch_size=32 
dataloader_train=DataLoader(dataset_train, 
                            sampler=RandomSampler(dataset_train), 
                            batch_size=batch_size)


In [ ]:
dataloader_test=DataLoader(dataset_test, 
                            sampler=RandomSampler(dataset_test), 
                            batch_size=batch_size)

####Setting the optimizer to Adam as a stochastic optimizer.



In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [ ]:
optimizer=AdamW(
    model.parameters(), 
    lr=1e-5, #2e-5 >5e-5
    eps=1e-8
)

In [ ]:
epochs=10
scheduler=get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=0, 
    #how many times learning rate is set to change
    num_training_steps=len(dataloader_train)*epochs
)

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import f1_score

####Predictions will come in the foramt of probabilities turned into 0s and 1s. 


In [ ]:
#preds=[0.9, 0.07, 0.05]--> #preds=[1, 0, 0]
def f1_score_func(preds, labels):
  preds_flat=np.argmax(preds, axis=1).flatten()
  labels_flat=labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [ ]:
def accuracy_per_class(preds, labels):
  label_dict_inverse={v: k for k, v in label_dict.items()}

  preds_flat=np.argmax(preds, axis=1).flatten()
  labels_flat=labels.flatten()

  for label in np.unique(labels_flat):
    y_preds=preds_flat[labels_flat==label]
    y_true=labels_flat[labels_flat==label]
    print(f'Class:{label_dict_inverse[label]}')
    print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')



In [ ]:
import random
seed_val=17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#send the model to the available device
model.to(device)
print(device)

cuda


In [ ]:
def evaluate(dataloader_test):
  model.eval()

  loss_test_total=0
  predictions, true_vals=[],[]
  
  for batch in dataloader_test:
      batch=tuple(b.to(device) for b in batch)

      inputs={
          'input_ids': batch[0],

          'attention_mask': batch[1],
          'labels': batch[2]

      }
      with torch.no_grad():
        outputs=model(**inputs)

      loss=outputs[0]
      logits= outputs[1]
      loss_test_total +=loss.item()
      #loss.backward()

      logits=logits.detach().cpu().numpy()
      label_ids=inputs['labels'].cpu().numpy()
      predictions.append(logits)
      true_vals.append(label_ids)



      
  loss_test_avg=loss_test_total/len(dataloader_test)

  predictions=np.concatenate(predictions, axis=0)
  true_vals=np.concatenate(true_vals, axis=0)


  return loss_test_avg, predictions, true_vals
  



In [ ]:
pip install tqdm


In [ ]:
from tqdm import tqdm

In [ ]:
for  epoch in tqdm(range(1, epochs+1)):  
  
  model.train()

  loss_train_total=0

  progress_bar=tqdm(dataloader_train, 
                    desc='Epoch {:1d}'.format(epoch), 
                    leave=False, 
                    disable=False)
                    
  for batch in progress_bar:
      model.zero_grad()
      batch=tuple(b.to(device) for b in batch)
      inputs={
          'input_ids': batch[0],

          'attention_mask': batch[1],
          'labels': batch[2]

      }
      #with torch.no_grad():
      outputs=model(**inputs)

      loss=outputs[0]
      loss_train_total +=loss.item()
      loss.backward()



      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()
      progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch) )})
  torch.save(model.state_dict(), f'/content/BERT_ft_epoch{epoch}.model')
  tqdm.write('\nEpoch {epoch}')
  loss_train_avg=loss_train_total/len(dataloader_train)
  tqdm.write(f'Training loss:{loss_train_avg}')
  val_loss, predictions, true_vals=evaluate(dataloader_test)

  val_f1=f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score (weighted): {val_f1}')






  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/493 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/493 [00:00<?, ?it/s, training_loss=0.226]

Epoch 1:   0%|          | 1/493 [00:00<03:52,  2.12it/s, training_loss=0.226]

Epoch 1:   0%|          | 1/493 [00:00<03:52,  2.12it/s, training_loss=0.263]

Epoch 1:   0%|          | 2/493 [00:00<03:48,  2.15it/s, training_loss=0.263]

Epoch 1:   0%|          | 2/493 [00:01<03:48,  2.15it/s, training_loss=0.193]

Epoch 1:   1%|          | 3/493 [00:01<03:46,  2.17it/s, training_loss=0.193]

Epoch 1:   1%|          | 3/493 [00:01<03:46,  2.17it/s, training_loss=0.188]

Epoch 1:   1%|          | 4/493 [00:01<03:43,  2.19it/s, training_loss=0.188]

Epoch 1:   1%|          | 4/493 [00:02<03:43,  2.19it/s, training_loss=0.112]

Epoch 1:   1%|          | 5/493 [00:02<03:41,  2.20it/s, training_loss=0.112]

Epoch 1:   1%|          | 5/493 [00:02<03:41,  2.20it/s, training_loss=0.234]

Epoch 1:   1%|          | 6/493 [00:02<03:40,  2


Epoch {epoch}
Training loss:0.6216764564023298




Epoch 1:   0%|          | 0/493 [58:56<?, ?it/s]

 10%|█         | 1/10 [04:05<36:45, 245.06s/it]

Epoch 2:   0%|          | 0/493 [00:00<?, ?it/s]

Validation loss: 1.3363022956442325
F1 Score (weighted): 0.6412352268761154




Epoch 2:   0%|          | 0/493 [00:00<?, ?it/s, training_loss=0.089]

Epoch 2:   0%|          | 1/493 [00:00<03:44,  2.19it/s, training_loss=0.089]

Epoch 2:   0%|          | 1/493 [00:00<03:44,  2.19it/s, training_loss=0.249]

Epoch 2:   0%|          | 2/493 [00:00<03:44,  2.19it/s, training_loss=0.249]

Epoch 2:   0%|          | 2/493 [00:01<03:44,  2.19it/s, training_loss=0.281]

Epoch 2:   1%|          | 3/493 [00:01<03:43,  2.19it/s, training_loss=0.281]

Epoch 2:   1%|          | 3/493 [00:01<03:43,  2.19it/s, training_loss=0.198]

Epoch 2:   1%|          | 4/493 [00:01<03:42,  2.19it/s, training_loss=0.198]

Epoch 2:   1%|          | 4/493 [00:02<03:42,  2.19it/s, training_loss=0.197]

Epoch 2:   1%|          | 5/493 [00:02<03:42,  2.20it/s, training_loss=0.197]

Epoch 2:   1%|          | 5/493 [00:02<03:42,  2.20it/s, training_loss=0.200]

Epoch 2:   1%|          | 6/493 [00:02<03:40,  2.21it/s, training_loss=0.200]

Epoch 2:   1%|          | 6/493 [00:03<03:40,  2.21it/s, t

...



Epoch 9:   0%|          | 0/493 [00:00<?, ?it/s, training_loss=0.240]

Epoch 9:   0%|          | 1/493 [00:00<03:46,  2.17it/s, training_loss=0.240]

Epoch 9:   0%|          | 1/493 [00:00<03:46,  2.17it/s, training_loss=0.173]

Epoch 9:   0%|          | 2/493 [00:00<03:44,  2.18it/s, training_loss=0.173]

Epoch 9:   0%|          | 2/493 [00:01<03:44,  2.18it/s, training_loss=0.204]

Epoch 9:   1%|          | 3/493 [00:01<03:43,  2.19it/s, training_loss=0.204]

Epoch 9:   1%|          | 3/493 [00:01<03:43,  2.19it/s, training_loss=0.218]

Epoch 9:   1%|          | 4/493 [00:01<03:42,  2.20it/s, training_loss=0.218]

Epoch 9:   1%|          | 4/493 [00:02<03:42,  2.20it/s, training_loss=0.322]

Epoch 9:   1%|          | 5/493 [00:02<03:43,  2.18it/s, training_loss=0.322]

Epoch 9:   1%|          | 5/493 [00:02<03:43,  2.18it/s, training_loss=0.334]

Epoch 9:   1%|          | 6/493 [00:02<03:42,  2.19it/s, training_loss=0.334]

Epoch 9:   1%|          | 6/493 [00:03<03:42,  2.19it/s, t


Epoch {epoch}
Training loss:0.6205555671187492




Epoch 1:   0%|          | 0/493 [1:31:35<?, ?it/s]

 90%|█████████ | 9/10 [36:44<04:04, 244.92s/it]

Epoch 10:   0%|          | 0/493 [00:00<?, ?it/s]

Validation loss: 1.3371314440213196
F1 Score (weighted): 0.6412352268761154




Epoch 10:   0%|          | 0/493 [00:00<?, ?it/s, training_loss=0.254]

Epoch 10:   0%|          | 1/493 [00:00<03:45,  2.18it/s, training_loss=0.254]

Epoch 10:   0%|          | 1/493 [00:00<03:45,  2.18it/s, training_loss=0.083]

Epoch 10:   0%|          | 2/493 [00:00<03:43,  2.19it/s, training_loss=0.083]

Epoch 10:   0%|          | 2/493 [00:01<03:43,  2.19it/s, training_loss=0.343]

Epoch 10:   1%|          | 3/493 [00:01<03:43,  2.20it/s, training_loss=0.343]

Epoch 10:   1%|          | 3/493 [00:01<03:43,  2.20it/s, training_loss=0.237]

Epoch 10:   1%|          | 4/493 [00:01<03:42,  2.20it/s, training_loss=0.237]

Epoch 10:   1%|          | 4/493 [00:02<03:42,  2.20it/s, training_loss=0.423]

Epoch 10:   1%|          | 5/493 [00:02<03:41,  2.21it/s, training_loss=0.423]

Epoch 10:   1%|          | 5/493 [00:02<03:41,  2.21it/s, training_loss=0.286]

Epoch 10:   1%|          | 6/493 [00:02<03:40,  2.21it/s, training_loss=0.286]

Epoch 10:   1%|          | 6/493 [00:03<03:40,

In [ ]:
model=BertForSequenceClassification.from_pretrained(
                'bert-base-uncased',
                 num_labels=19, 
                  output_attentions=False, 
                 output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

####We choose the model from the last epoch, here the 8th epoch.

In [ ]:
model.load_state_dict(torch.load('/content/BERT_ft_epoch10.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)
print(device)

cuda


In [ ]:
_, prediction, true_vals= evaluate(dataloader_test)

####F1 score

In [ ]:
f1_score_func(prediction, true_vals)

0.6412352268761154

In [ ]:
accuracy_per_class(prediction, true_vals)

Class:O
Accuracy:2746/3000

Class:Black-and-White_Fallacy
Accuracy:0/26

Class:Loaded_Language
Accuracy:247/436

Class:Flag-Waving
Accuracy:25/96

Class:Name_Calling,Labeling
Accuracy:50/209

Class:Slogans
Accuracy:0/36

Class:Causal_Oversimplification
Accuracy:0/33

Class:Whataboutism
Accuracy:0/25

Class:Exaggeration,Minimisation
Accuracy:4/99

Class:Doubt
Accuracy:7/89

Class:Appeal_to_Authority
Accuracy:0/56

Class:Repetition
Accuracy:6/195

Class:Appeal_to_fear-prejudice
Accuracy:1/130

Class:Thought-terminating_Cliches
Accuracy:0/22

Class:Bandwagon
Accuracy:0/4

Class:Red_Herring
Accuracy:0/15

Class:Reductio_ad_hitlerum
Accuracy:0/11

Class:Obfuscation,Intentional_Vagueness,Confusion
Accuracy:0/6

Class:Straw_Men
Accuracy:0/2



In [ ]:
print(len(prediction))
assert(len(predictions)==len(df_test))

4490


In [ ]:
a=prediction[4000]
print("an example of the predicted vectors is: ", a)

an example of the predicted vectors is:  [ 7.7178917  -1.1620193   0.5262173  -1.5242306  -0.3068546  -1.0497875
 -0.7470739  -1.6815195  -0.6961153   0.5543403   0.302957    0.562094
 -0.5063292  -0.36377698 -1.7534236  -1.2366631  -1.8603466  -1.4071819
 -1.6673759 ]


In [ ]:
pred=[]
for i in range (len(prediction)):
  pred.append(np.argmax(prediction[i]).flatten())


In [ ]:
print(pred[9])

[0]


In [ ]:
print(len(true_vals))

4490


In [ ]:
print(true_vals[1000])

1


In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score

#predicted = [1,2,3,4,5,1,2,1,1,4,5] 
#y_test = [1,2,3,4,5,1,2,1,1,4,1]

precision, recall, fscore, support = score(true_vals, pred)

#print('precision: {}'.format(precision))
#print('recall: {}'.format(recall))
#print('fscore: {}'.format(fscore))
#print('support: {}'.format(support))



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
df1 = pd.DataFrame({ 'precision': precision, 'recall':recall, 'Fscore':fscore, 'support':support })


In [ ]:
df1.head


<bound method NDFrame.head of     precision    recall    Fscore  support
0    0.810269  0.915333  0.859602     3000
1    0.000000  0.000000  0.000000       26
2    0.352354  0.566514  0.434477      436
3    0.301205  0.260417  0.279330       96
4    0.340136  0.239234  0.280899      209
5    0.000000  0.000000  0.000000       36
6    0.000000  0.000000  0.000000       33
7    0.000000  0.000000  0.000000       25
8    0.235294  0.040404  0.068966       99
9    0.082353  0.078652  0.080460       89
10   0.000000  0.000000  0.000000       56
11   0.120000  0.030769  0.048980      195
12   0.090909  0.007692  0.014184      130
13   0.000000  0.000000  0.000000       22
14   0.000000  0.000000  0.000000        4
15   0.000000  0.000000  0.000000       15
16   0.000000  0.000000  0.000000       11
17   0.000000  0.000000  0.000000        6
18   0.000000  0.000000  0.000000        2>

In [ ]:
from tabulate import tabulate

print(tabulate(df1, headers='keys', tablefmt='psql'))


+----+-------------+------------+-----------+-----------+
|    |   precision |     recall |    Fscore |   support |
|----+-------------+------------+-----------+-----------|
|  0 |   0.810269  | 0.915333   | 0.859602  |      3000 |
|  1 |   0         | 0          | 0         |        26 |
|  2 |   0.352354  | 0.566514   | 0.434477  |       436 |
|  3 |   0.301205  | 0.260417   | 0.27933   |        96 |
|  4 |   0.340136  | 0.239234   | 0.280899  |       209 |
|  5 |   0         | 0          | 0         |        36 |
|  6 |   0         | 0          | 0         |        33 |
|  7 |   0         | 0          | 0         |        25 |
|  8 |   0.235294  | 0.040404   | 0.0689655 |        99 |
|  9 |   0.0823529 | 0.0786517  | 0.0804598 |        89 |
| 10 |   0         | 0          | 0         |        56 |
| 11 |   0.12      | 0.0307692  | 0.0489796 |       195 |
| 12 |   0.0909091 | 0.00769231 | 0.0141844 |       130 |
| 13 |   0         | 0          | 0         |        22 |
| 14 |   0    